# HW 1 

MUSA 650

Lu Yii Wong

31 Jan 2025

In [1]:
import pandas as pandas
import numpy as np
import matplotlib.pyplot as plt
import keras

from keras.datasets import mnist


(x_train, y_train), (x_test, y_test) = mnist.load_data()